In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## import 用到的套件

In [2]:
import numpy as np
#載入繪圖套件
import matplotlib.pyplot as plt
import matplotlib.font_manager as plt_font
#載入for迴圈進度條
from tqdm import tqdm
folder="/content/drive/MyDrive/解密AI黑盒子分享/"
twfont1 = plt_font.FontProperties(fname=folder+"字型/kaiu.ttf")

In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,models,datasets
from torch.utils.data import DataLoader

In [4]:
import numpy as np
import pandas as pd
import time
import torch
import torch.nn.functional as F
from torchvision import transforms,models
import matplotlib.pyplot as plt
import matplotlib.font_manager as plt_font
folder="/content/drive/MyDrive/解密AI黑盒子分享/"
twfont1 = plt_font.FontProperties(fname=folder+"字型/kaiu.ttf")
import sys
sys.path.append(folder+"模組/")
import webcam

## 是否有GPU可以使用

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 定義訓練集和測試集資料預處理

In [6]:
train_transform=transforms.Compose([
    transforms.RandomResizedCrop(224),#隨機裁剪 #資料增強
    transforms.RandomHorizontalFlip(),#隨機旋轉 #資料增強
    transforms.ToTensor(),#轉成Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])#像素資料歸一化
])

test_transform=transforms.Compose([
    transforms.Resize(256),#改變大小
    transforms.CenterCrop(224),#中央裁剪
    transforms.ToTensor(),#轉成Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])#像素資料歸一化
])

## 載入訓練和測試資料

In [7]:
train_dataset = datasets.ImageFolder("/content/drive/MyDrive/專題--電子元件的辨識/照片(接頭)/train", train_transform)
test_dataset = datasets.ImageFolder("/content/drive/MyDrive/專題--電子元件的辨識/照片(接頭)/test", test_transform)

In [8]:
print('訓練集圖片資料', len(train_dataset))
print('測試集圖片資料', len(test_dataset))
print('分類名稱', train_dataset.classes)
print('分類數量', len(train_dataset.classes))
n_class=len(train_dataset.classes) #等等才知道輸出層要改幾個節點

訓練集圖片資料 367
測試集圖片資料 105
分類名稱 ['HDMI', 'Micro USB', 'Mini USB', 'Type-A', 'Type-C', 'VGA', 'lightning']
分類數量 7


## 分類和索引雙向對應，並存檔

In [9]:
index2label = {i:x for i,x in enumerate(train_dataset.classes) }
print(index2label)
np.save("/content/drive/MyDrive/專題--電子元件的辨識/model/接頭.npy", index2label)

{0: 'HDMI', 1: 'Micro USB', 2: 'Mini USB', 3: 'Type-A', 4: 'Type-C', 5: 'VGA', 6: 'lightning'}


## 定義數據加載DataLoader

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=2)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=2)

## 遷移式學習方法

### 方法一：只訓練最後一層全連階層

In [ ]:
#載入預訓練模型
model=models.resnet18(weights='DEFAULT')
#更改最後一層全連階層
model.fc=nn.Linear(model.fc.in_features,n_class)
#設定優化器的更新範圍
optimizer = torch.optim.Adam(model.fc.parameters()) #設定優化範圍(這裡只微調輸出層)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 71.0MB/s]


### 方法二：微調訓練所有層的參數

In [ ]:
#載入預訓練模型
model = models.resnet18(weights='DEFAULT')
#更改最後一層全連階層
model.fc = nn.Linear(model.fc.in_features,n_class)
#設定優化器的更新範圍
optimizer = torch.optim.Adam(model.parameters()) #微調所有參數

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 125MB/s]


### 方法三：從頭訓練所有層的參數

In [ ]:
#載入預訓練模型
model=models.resnet18() #花最久時間
#更改最後一層全連階層
model.fc=nn.Linear(model.fc.in_features,n_class)
#設定優化器的更新範圍
optimizer=torch.optim.Adam(model.parameters())

In [ ]:
# model

## 模型訓練的配置

In [ ]:
model=model.to(device)
LossFun=nn.CrossEntropyLoss()

## 開始執行訓練

In [ ]:
for epoch in tqdm(range(30)): #tqdm >> 產生進度條
  model.train()
  for images, labels in train_loader:
    optimizer.zero_grad()
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = LossFun(outputs, labels)
    loss.backward()
    optimizer.step()

100%|██████████| 30/30 [01:37<00:00,  3.24s/it]


## 用測試集資料進行測試

In [ ]:
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    correct += (torch.argmax(outputs,dim=1)==labels).sum().item()
  print("驗證資料準確度：",correct*100/len(test_dataset),"%")

100%|██████████| 4/4 [00:00<00:00,  5.33it/s]

驗證資料準確度： 35.23809523809524 %


## 保存模型

In [ ]:
torch.save(model, "/content/drive/MyDrive/專題--電子元件的辨識/model/接頭.pth")